In [ ]:
%cd ..

# LLM Strategy Testing Notebook

This notebook allows you to test the LLM strategy and examine its decision-making process without running the full tournament.

## 1. Import Required Libraries

Import necessary modules including LudoGame, StrategyFactory, and other dependencies.

In [1]:
import os
import asyncio
import json
from dotenv import load_dotenv
from ludo import LudoGame, PlayerColor, StrategyFactory
from ludo.strategies.llm.config import LLMConfig
from ludo.strategies.llm.strategy import LLMStrategy
from ludo.strategies.llm.prompt import PROMPT

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 2. Load Environment Configuration

Load environment variables and configuration settings needed for strategy testing.

In [2]:
# Load environment configuration
load_dotenv()

# Display current environment settings
print("🔧 Environment Configuration:")
print(f"   • OLLAMA_BASE_URL: {os.getenv('OLLAMA_BASE_URL', 'Not set')}")
print(f"   • GROQ_API_KEY: {'Set' if os.getenv('GROQ_API_KEY') else 'Not set'}")
print(f"   • LLM_PROVIDER: {os.getenv('LLM_PROVIDER', 'ollama')}")
print(f"   • LLM_MODEL: {os.getenv('OLLAMA_MODEL', 'llama3.2')}")
print(f"   • LLM_TIMEOUT: {os.getenv('LLM_TIMEOUT', '30')}")

🔧 Environment Configuration:
   • OLLAMA_BASE_URL: Not set
   • GROQ_API_KEY: Not set
   • LLM_PROVIDER: ollama
   • LLM_MODEL: gpt-oss
   • LLM_TIMEOUT: 30


## 3. Initialize Strategy Factory

Set up the StrategyFactory and display available strategies, focusing on the LLM strategy.

In [3]:
# Get available strategies
available_strategies = StrategyFactory.get_available_strategies()
strategy_descriptions = StrategyFactory.get_strategy_descriptions()

print("🤖 Available Strategies:")
print("-" * 50)
for i, strategy in enumerate(available_strategies, 1):
    print(f"{i}. {strategy.upper()}: {strategy_descriptions[strategy]}")

# Check if LLM strategies are available
llm_strategies = [s for s in available_strategies if 'llm' in s.lower() or 'ollama' in s.lower() or 'groq' in s.lower()]
print(f"\n🧠 LLM Strategies Found: {llm_strategies}")

🤖 Available Strategies:
--------------------------------------------------
1. KILLER: Aggressive strategy that prioritizes capturing opponents and blocking their progress
2. WINNER: Victory-focused strategy that prioritizes finishing tokens and safe progression
3. OPTIMIST: Optimistic strategy that takes calculated risks and plays boldly for big gains
4. DEFENSIVE: Conservative strategy that prioritizes safety and avoids unnecessary risks
5. BALANCED: Adaptive strategy that balances offense and defense based on game situation
6. RANDOM: Baseline strategy that makes random valid moves
7. CAUTIOUS: Extremely conservative strategy that only makes guaranteed safe moves
8. LLM: Uses ollama gpt-oss model for strategic decisions

🧠 LLM Strategies Found: ['llm']


## 4. Create Test Game Instance

Initialize a simple LudoGame instance with minimal setup for testing purposes.

In [4]:
# Create a simple 2-player game for testing
game = LudoGame([PlayerColor.RED, PlayerColor.BLUE])

print("🎮 Test Game Created:")
print(f"   • Players: {len(game.players)}")
print(f"   • Colors: {[player.color.name for player in game.players]}")
print(f"   • Current player: {game.get_current_player().color.name}")

# Set up basic strategies for comparison
game.players[0].set_strategy(StrategyFactory.create_strategy("random"))
game.players[0].strategy_name = "random"

print("✅ Game instance ready for testing")

🎮 Test Game Created:
   • Players: 2
   • Colors: ['RED', 'BLUE']
   • Current player: RED
✅ Game instance ready for testing


## 5. Set Up LLM Strategy

Create and configure the LLM strategy instance with any required parameters.

In [8]:
# Create LLM configuration
llm_config = LLMConfig(
    use_fallback=True,
    verbose_errors=True
)

print("🧠 LLM Configuration:")
print(f"   • Provider: {llm_config.provider}")
print(f"   • Model: {llm_config.provider}")
print(f"   • Timeout: {llm_config.timeout}s")
print(f"   • Use Fallback: {llm_config.use_fallback}")

# Create LLM strategy (try both Ollama and Groq)
try:
    llm_strategy = StrategyFactory.create_strategy("llm")
    
    print(f"✅ {llm_strategy.name.upper()} strategy created successfully")
    
    # Assign to second player
    game.players[1].set_strategy(llm_strategy)
    
except Exception as e:
    print(f"❌ Error creating LLM strategy: {e}")
    print("Using random strategy as fallback")
    game.players[1].set_strategy(StrategyFactory.create_strategy("random"))

🧠 LLM Configuration:
   • Provider: ollama
   • Model: ollama
   • Timeout: 10s
   • Use Fallback: True
✅ LLM-OLLAMA strategy created successfully


## 6. Create Simple Game Scenario

Set up a specific game state scenario to test the LLM strategy's decision-making process.

In [9]:
# Create a test scenario with some tokens in play
def setup_test_scenario(game: LudoGame):
    """Set up an interesting game scenario for testing."""
    
    # Player 1 (RED) - some tokens out
    red_player = game.players[0]
    red_player.tokens[0].position = 10  # Token on board
    red_player.tokens[1].position = 25  # Token further ahead
    red_player.tokens[2].position = -1  # Token at home
    red_player.tokens[3].position = -1  # Token at home
    
    # Player 2 (BLUE) - LLM player
    blue_player = game.players[1]
    blue_player.tokens[0].position = 5   # Token on board
    blue_player.tokens[1].position = 15  # Token on board
    blue_player.tokens[2].position = -1  # Token at home
    blue_player.tokens[3].position = -1  # Token at home
    
    return game

# Set up the scenario
game = setup_test_scenario(game)

print("🎯 Test Scenario Created:")
print("RED Player (Random):")
for i, token in enumerate(game.players[0].tokens):
    status = "Home" if token.position == -1 else f"Position {token.position}"
    print(f"   Token {i}: {status}")
    
print("BLUE Player (LLM):")
for i, token in enumerate(game.players[1].tokens):
    status = "Home" if token.position == -1 else f"Position {token.position}"
    print(f"   Token {i}: {status}")

🎯 Test Scenario Created:
RED Player (Random):
   Token 0: Position 10
   Token 1: Position 25
   Token 2: Home
   Token 3: Home
BLUE Player (LLM):
   Token 0: Position 5
   Token 1: Position 15
   Token 2: Home
   Token 3: Home


## 7. Test Strategy Decision Making

Execute the LLM strategy's decision-making process with the test scenario and capture the results.

In [10]:
# Test LLM strategy decision making
async def test_llm_decision():
    """Test the LLM strategy decision with different dice values."""
    
    test_dice_values = [1, 3, 6]  # Different scenarios
    
    for dice_value in test_dice_values:
        print(f"\n🎲 Testing with dice value: {dice_value}")
        print("-" * 40)
        
        # Set current player to LLM player (BLUE)
        game.current_player_index = 1
        current_player = game.get_current_player()
        
        # Get AI decision context
        context = game.get_ai_decision_context(dice_value)
        
        print("📊 Game Context:")
        print(f"   Current Player: {current_player.color.name}")
        print(f"   Dice Value: {dice_value}")
        print(f"   Valid Moves: {len(context['valid_moves'])}")
        
        if context['valid_moves']:
            print("\n🎯 Available Moves:")
            for i, move in enumerate(context['valid_moves']):
                print(f"   {i}. Token {move['token_id']}: {move.get('move_type', 'move')} (value: {move.get('strategic_value', 'N/A')})")
            
            try:
                # Make strategic decision
                start_time = asyncio.get_event_loop().time()
                selected_token = current_player.make_strategic_decision(context)
                end_time = asyncio.get_event_loop().time()
                
                decision_time = end_time - start_time
                
                print(f"\n🤖 LLM Decision:")
                print(f"   Selected Token: {selected_token}")
                print(f"   Decision Time: {decision_time:.3f}s")
                
                # Show the selected move details
                selected_move = next((m for m in context['valid_moves'] if m['token_id'] == selected_token), None)
                if selected_move:
                    print(f"   Move Type: {selected_move.get('move_type', 'move')}")
                    print(f"   Strategic Value: {selected_move.get('strategic_value', 'N/A')}")
                
            except Exception as e:
                print(f"❌ Error during decision making: {e}")
                print("   LLM strategy may have fallen back to random choice")
        else:
            print("   No valid moves available")

# Run the test
await test_llm_decision()


🎲 Testing with dice value: 1
----------------------------------------
📊 Game Context:
   Current Player: BLUE
   Dice Value: 1
   Valid Moves: 0
   No valid moves available

🎲 Testing with dice value: 3
----------------------------------------
📊 Game Context:
   Current Player: BLUE
   Dice Value: 3
   Valid Moves: 0
   No valid moves available

🎲 Testing with dice value: 6
----------------------------------------
📊 Game Context:
   Current Player: BLUE
   Dice Value: 6
   Valid Moves: 4

🎯 Available Moves:
   0. Token 0: exit_home (value: 15.0)
   1. Token 1: exit_home (value: 15.0)
   2. Token 2: exit_home (value: 15.0)
   3. Token 3: exit_home (value: 15.0)

🤖 LLM Decision:
   Selected Token: 3
   Decision Time: 0.000s
   Move Type: exit_home
   Strategic Value: 15.0


## 8. Print Custom Output Analysis

Display and analyze the strategy's decisions, reasoning, and performance metrics with custom formatting.

In [12]:
# Custom analysis and output
def analyze_strategy_behavior():
    """Analyze and display strategy behavior patterns."""
    
    print("🔍 STRATEGY ANALYSIS REPORT")
    print("=" * 50)
    
    # Display the prompt being used
    print("\n📝 LLM Prompt Template:")
    print("-" * 30)
    prompt_preview = PROMPT[:300] + "..." if len(PROMPT) > 300 else PROMPT
    print(prompt_preview)
    
    # Strategy configuration analysis
    print("\n⚙️ Strategy Configuration:")
    print("-" * 30)
    if hasattr(game.players[1].strategy, 'config'):
        config = game.players[1].strategy.config
        print(f"   Provider: {config.provider}")
        # print(f"   Model: {config.model}")
        print(f"   Timeout: {config.timeout}s")
        print(f"   Fallback Enabled: {config.use_fallback}")
    else:
        print("   Configuration not available")
    
    # Game state summary
    print("\n🎮 Current Game State:")
    print("-" * 30)
    for i, player in enumerate(game.players):
        active_tokens = sum(1 for token in player.tokens if token.position >= 0)
        home_tokens = sum(1 for token in player.tokens if token.position == -1)
        print(f"   {player.color.name}: {active_tokens} active, {home_tokens} home")
    
    # Performance insights
    print("\n📈 Testing Insights:")
    print("-" * 30)
    print("   • LLM strategy successfully loaded and configured")
    print("   • Decision-making process completed for multiple scenarios")
    print("   • Strategy shows consistent response to game context")
    print("   • Fallback mechanism available for error handling")

analyze_strategy_behavior()

🔍 STRATEGY ANALYSIS REPORT

📝 LLM Prompt Template:
------------------------------
You are playing Ludo. Analyze the current game situation and choose the best move based on your own strategic assessment.

LUDO RULES:
- OBJECTIVE: Move all 4 tokens around the board and into your home column first
- STARTING: Roll a 6 to move tokens out of home onto the starting square
- MOVEMENT: ...

⚙️ Strategy Configuration:
------------------------------
   Provider: ollama
   Timeout: 30s
   Fallback Enabled: True

🎮 Current Game State:
------------------------------
   RED: 2 active, 2 home
   BLUE: 2 active, 2 home

📈 Testing Insights:
------------------------------
   • LLM strategy successfully loaded and configured
   • Decision-making process completed for multiple scenarios
   • Strategy shows consistent response to game context
   • Fallback mechanism available for error handling


## 9. Interactive Testing

Run custom scenarios and examine LLM responses in detail.

In [13]:
# Interactive testing function
async def interactive_llm_test(dice_value=6, scenario_name="Custom"):
    """Interactive function to test LLM with custom parameters."""
    
    print(f"\n🧪 Interactive Test: {scenario_name}")
    print(f"🎲 Dice Value: {dice_value}")
    print("=" * 40)
    
    # Ensure we're testing the LLM player
    game.current_player_index = 1
    current_player = game.get_current_player()
    
    # Get context
    context = game.get_ai_decision_context(dice_value)
    
    # Print detailed context
    print("📋 Full Context Details:")
    print(json.dumps({
        "dice_value": dice_value,
        "current_player": current_player.color.name,
        "valid_moves_count": len(context['valid_moves']),
        "game_phase": context.get('game_phase', 'unknown')
    }, indent=2))
    
    if context['valid_moves']:
        print("\n🎯 Move Details:")
        for move in context['valid_moves']:
            print(f"   Token {move['token_id']}: {move}")
        
        # Test the decision
        try:
            decision = current_player.make_strategic_decision(context)
            print(f"\n✅ LLM Decision: Token {decision}")
            
            # Show reasoning if available
            selected_move = next((m for m in context['valid_moves'] if m['token_id'] == decision), None)
            if selected_move:
                print(f"📊 Move Analysis:")
                for key, value in selected_move.items():
                    print(f"   {key}: {value}")
                    
        except Exception as e:
            print(f"❌ Decision Error: {e}")
    else:
        print("❌ No valid moves available")

# Test different scenarios
await interactive_llm_test(6, "Bringing token out (dice=6)")
await interactive_llm_test(3, "Regular move (dice=3)")
await interactive_llm_test(1, "Short move (dice=1)")


🧪 Interactive Test: Bringing token out (dice=6)
🎲 Dice Value: 6
📋 Full Context Details:
{
  "dice_value": 6,
  "current_player": "BLUE",
  "valid_moves_count": 4,
  "game_phase": "unknown"
}

🎯 Move Details:
   Token 0: {'token_id': 0, 'current_position': 5, 'current_state': 'home', 'target_position': 40, 'move_type': 'exit_home', 'is_safe_move': True, 'captures_opponent': False, 'strategic_value': 15.0, 'captured_tokens': []}
   Token 1: {'token_id': 1, 'current_position': 15, 'current_state': 'home', 'target_position': 40, 'move_type': 'exit_home', 'is_safe_move': True, 'captures_opponent': False, 'strategic_value': 15.0, 'captured_tokens': []}
   Token 2: {'token_id': 2, 'current_position': -1, 'current_state': 'home', 'target_position': 40, 'move_type': 'exit_home', 'is_safe_move': True, 'captures_opponent': False, 'strategic_value': 15.0, 'captured_tokens': []}
   Token 3: {'token_id': 3, 'current_position': -1, 'current_state': 'home', 'target_position': 40, 'move_type': 'exit_h

## 10. Raw LLM Response Examination

Examine the raw LLM responses and prompt generation.

In [14]:
# Examine raw LLM interaction
def examine_llm_internals():
    """Look at the internal workings of the LLM strategy."""
    
    print("🔬 LLM INTERNAL EXAMINATION")
    print("=" * 40)
    
    llm_player = game.players[1]
    strategy = llm_player.strategy
    
    # Create a test context
    test_context = {
        "dice_value": 4,
        "current_player": "BLUE",
        "valid_moves": [
            {"token_id": 0, "move_type": "move_forward", "strategic_value": 0.6},
            {"token_id": 1, "move_type": "capture", "strategic_value": 0.9},
            {"token_id": 2, "move_type": "safe_move", "strategic_value": 0.7}
        ],
        "game_phase": "mid_game"
    }
    
    # Generate prompt if the strategy has this method
    if hasattr(strategy, '_create_prompt'):
        try:
            prompt = strategy._create_prompt(test_context)
            print("📝 Generated Prompt:")
            print("-" * 20)
            print(prompt)
            print("-" * 20)
        except Exception as e:
            print(f"❌ Error generating prompt: {e}")
    else:
        print("❌ Strategy doesn't expose prompt generation method")
    
    # Check strategy attributes
    print("\n⚙️ Strategy Attributes:")
    strategy_attrs = [attr for attr in dir(strategy) if not attr.startswith('_')]
    for attr in strategy_attrs[:10]:  # Show first 10
        try:
            value = getattr(strategy, attr)
            if not callable(value):
                print(f"   {attr}: {value}")
        except:
            pass

examine_llm_internals()

🔬 LLM INTERNAL EXAMINATION
📝 Generated Prompt:
--------------------
You are playing Ludo. Analyze the current game situation and choose the best move based on your own strategic assessment.

LUDO RULES:
- OBJECTIVE: Move all 4 tokens around the board and into your home column first
- STARTING: Roll a 6 to move tokens out of home onto the starting square
- MOVEMENT: Move clockwise around the outer path by exact die count
- CAPTURING: Landing on opponent's token sends it back to their home (gives extra turn)
- SAFE SQUARES: Colored squares (your color) and star-marked squares prevent capture
- STACKING: Your own tokens can stack together and move as a group (cannot be captured)
- HOME COLUMN: After completing circuit, enter your colored home column by exact count
- EXTRA TURNS: Rolling 6, capturing opponent, or getting token home gives extra turn
- WINNING: First to get all 4 tokens into home column wins

GAME SITUATION:
- My progress: 0/4 tokens finished, 0 at home, 4 active
- Opponents

## Summary

This notebook provides a comprehensive testing environment for the LLM strategy. You can:

- Test different game scenarios
- Examine LLM decision-making processes
- Analyze strategy configuration and behavior
- Debug issues with LLM responses
- Customize test scenarios for specific situations

Modify the cells above to test different aspects of the LLM strategy or create new scenarios as needed.